In [234]:
import csv
import pandas as pd
import numpy as np
import sklearn

from pymongo import MongoClient

ALL_COLORS = ['W', 'U', 'B', 'R', 'G']

PATH = './data/draft_data_public.MKM.PremierDraft.csv'

NO_NO = 0
NO_YES = 1
YES_YES = 2

NO = 0
YES = 1

MAX_PIP_COUNT = 40
NUM_COLORS = 5
NUM_IN_PACK = 13
NUM_PACKS = 3

In [340]:
client = MongoClient()
cards_en = client.scryfall.cards_en

with open(PATH) as csvfile:
    draft_data = csv.reader(csvfile)
    COLUMNS = next(draft_data)

NAMES = [col.split('pack_card_')[1] for col in columns if 'pack_card_' in col]
PICK_IDX = columns.index('pick')
DRAFTID_IDX = columns.index('draft_id')
DRAFTTIME_IDX = columns.index('draft_time')
PACKNUM_IDX = columns.index('pack_number')
PICKNUM_IDX = columns.index('pick_number')
PACK_IDX = columns.index('pack_card_A Killer Among Us')
POOL_IDX = columns.index('pool_A Killer Among Us')
NUM_CARDS = len(names)


In [644]:
def get_card(card_name):
    card = cards_en.find_one({'name': card_name, 'set_type': {'$in': ['masters', 'expansion', 'masterpiece']}})

    if card is None:
        query = {            
            'card_faces': {
                '$elemMatch': {
                    'name': card_name
                }
            },
        }

        card = cards_en.find_one(query)

    if card is None:
        error_message = 'No match for card name {}'.format(card_name)
        print(error_message)

    return(card)

def card_vector(index, row):
    return np.array([int(x) for x in row[index:index + NUM_CARDS]])

In [565]:
alsa_filepath = './data/alsa.npy'
# calculate ALSA

last_seen = np.zeros(len(names))
counts = np.zeros(len(names))
skipped_count = 0
with open(PATH) as csvfile:
    draft_data = csv.reader(csvfile)
    next(draft_data)
    draft_id = None
    pack_num = None
    row_last_seen = np.zeros(len(names))
    row = next(draft_data)
    
    while(row[DRAFTTIME_IDX] < '2024-02-20'):
        skipped_count += 1
        row = next(draft_data)
            
    while(row and row[DRAFTTIME_IDX] < '2024-04-01'):
        if draft_id != row[DRAFTID_IDX] or pack_num != row[PACKNUM_IDX]:
            last_seen += row_last_seen
            counts += np.where(row_last_seen, 1, 0)
            draft_id = row[DRAFTID_IDX]
            pack_num = int(row[PACKNUM_IDX])
            row_last_seen = np.zeros(NUM_CARDS)
        pack = card_vector(PACK_IDX, row)
        np.putmask(row_last_seen, pack, int(row[PICKNUM_IDX]))
        try:
            row = next(draft_data)
        except:
            row = None
        
alsa = last_seen / counts
alsa_series = pd.Series(alsa, index=NAMES)
alsa_series = alsa_series[pd.notna(alsa)]

with open(alsa_filepath, 'wb') as f:
    np.save(f, alsa)

# alsa = np.load(alsa_filepath)

StopIteration: 

In [577]:
alsa = last_seen / counts
alsa_series = pd.Series(alsa, index=NAMES)
alsa_series = alsa_series[pd.notna(alsa)]

/tmp/ipykernel_29055/3642633259.py:1: RuntimeWarning: invalid value encountered in divide
  alsa = last_seen / counts


In [645]:
def pip_vector(card_name):
    card = get_card(card_name)
    if card is None:
        print(card_name + ' Not Found')
    mana_cost = card['mana_cost']
    return np.array([mana_cost.count(f'{{{color}}}') for color in ALL_COLORS])  \
        + 1/2 * np.array([mana_cost.count(f'/{color}') for color in ALL_COLORS])\
        + 1/2 * np.array([mana_cost.count(f'{color}/') for color in ALL_COLORS])

PIP_LOOKUP = np.array([pip_vector(name) for name in names])

def pool_pips(pool):
    return np.matmul(pool.reshape(1,NUM_CARDS), PIP_LOOKUP)[0]

def pick_number(row):
    return int(row[PICKNUM_IDX]) + NUM_IN_PACK * int(row[PACKNUM_IDX])

def pip_counts_to_matrix(pip_counts):
    matrix = np.zeros((NUM_COLORS, MAX_PIP_COUNT), dtype=np.intc)
    for i in range(NUM_COLORS):
        matrix[i][0:int(pip_counts[i])] = 1
    return matrix
       


In [590]:
COLOR_NAMES = ['White', 'Blue', 'Black', 'Red', 'Green']

def print_pack(row):
    print( "========" )
    print( f"Draft Id: {row[DRAFTID_IDX]}, Pack {row[PACKNUM_IDX]}, Pick {row[PICKNUM_IDX]}")
    print( "--------" )
    pack_vector = card_vector(PACK_IDX, row)
    if (pack_vector.sum() > 1):
        pack_alsa = np.where(pack_vector, alsa, NUM_IN_PACK + 1)
        min_indices = np.argpartition(pack_alsa, 2)
        for idx in min_indices[0:2]:
            print( names[idx] + f" (ALSA: {pack_alsa[idx]:.2f})" )
        print( f"And {pack_vector.sum() - 2} other cards" )
    else:
        print( names[pack_vector.argmax()] )
    print("--------")
    print(f"Picked: {row[PICK_IDX]}")
    print("========")
    print("")
        
def print_pool_dist(pool_matrix):
    pips = pool_matrix.sum(axis=1)
    total = pips.sum()
    args = np.flip(pips.argsort())
    print("********")
    for color in args:
        print( ALL_COLORS[color]*round((pips[color]))  + f" ({pips[color]:.2f} {COLOR_NAMES[color]})")
    print(f'{total:.2f} total')
    print("********")
    print("")
            

In [646]:
experiment_counts = np.zeros((NUM_PACKS * NUM_IN_PACK, len([NO_NO, NO_YES, YES_YES]), NUM_COLORS, MAX_PIP_COUNT), dtype=np.intc)
unseen_events = np.zeros((NUM_PACKS * NUM_IN_PACK, len([NO_NO, NO_YES, YES_YES]), NUM_COLORS, MAX_PIP_COUNT), dtype=np.intc)
sender_events = np.zeros((NUM_PACKS * NUM_IN_PACK, NUM_CARDS, len([NO_NO, NO_YES, YES_YES]), NUM_COLORS, MAX_PIP_COUNT), dtype=np.intc)
receiver_events = np.zeros((NUM_PACKS * NUM_IN_PACK, NUM_CARDS, len([NO_NO, NO_YES, YES_YES]), NUM_COLORS, MAX_PIP_COUNT), dtype=np.intc) 
test_rows = []

with open(PATH) as csvfile:
    draft_data = csv.reader(csvfile)
    next(draft_data) # discard names
    row = next(draft_data)
    
    while(row[DRAFTTIME_IDX] < '2024-02-20'):
        row = next(draft_data)

    while(row[DRAFTTIME_IDX] < '2024-03-18'):
        pack = card_vector(PACK_IDX, row) 
        pack_alsa = np.where(pack, alsa, NUM_IN_PACK+1)
        pool = card_vector(POOL_IDX, row)
        pool_pip_counts_pre = pool_pips(pool)
        pool_pre_matrix = pip_counts_to_matrix(pool_pip_counts_pre)
        pool_pip_counts_post = pool_pip_counts_pre + PIP_LOOKUP[names.index(row[PICK_IDX])]

        experiment_counts[pick_number(row), NO_NO] += 1 - pool_pre_matrix
        experiment_counts[pick_number(row), NO_YES] += 1 - pool_pre_matrix
        experiment_counts[pick_number(row), YES_YES] += pool_pre_matrix

        pool_post_matrix = pip_counts_to_matrix(pool_pip_counts_post)
                
        no_no_matrix = 1 - pool_post_matrix
        no_yes_matrix = pool_post_matrix - pool_pre_matrix

        unseen_events[pick_number(row), NO_NO] += no_no_matrix
        unseen_events[pick_number(row), NO_YES] += no_yes_matrix
        unseen_events[pick_number(row), YES_YES] += pool_pre_matrix
        
        min_indices = np.argpartition(pack_alsa, 1)
        
        receiver_events[pick_number(row), min_indices[0], NO_NO] += no_no_matrix
        receiver_events[pick_number(row), min_indices[0], NO_YES] += no_yes_matrix
        receiver_events[pick_number(row), min_indices[0], YES_YES] += pool_pre_matrix
        
        top_passed_card = min_indices[1] if names[min_indices[0]] == row[columns.index('pick')] else min_indices[0]
            
        sender_events[pick_number(row), top_passed_card, NO_NO] += no_no_matrix
        sender_events[pick_number(row), top_passed_card, NO_YES] += no_yes_matrix
        sender_events[pick_number(row), top_passed_card, YES_YES] += pool_pre_matrix

        row = next(draft_data)

    test_rows.append(row)
    for row in draft_data:
        test_rows.append(row)

sender_parameters = np.nan_to_num(sender_events / experiment_counts[:, np.newaxis, ...], 0)
receiver_parameters = np.nan_to_num(receiver_events / experiment_counts[:, np.newaxis, ...], 0)
unseen_parameters = np.nan_to_num(unseen_events / experiment_counts, 0)      

/tmp/ipykernel_29055/1728990570.py:54: RuntimeWarning: invalid value encountered in divide
  sender_parameters = np.nan_to_num(sender_events / experiment_counts[:, np.newaxis, ...], 0)
/tmp/ipykernel_29055/1728990570.py:55: RuntimeWarning: invalid value encountered in divide
  receiver_parameters = np.nan_to_num(receiver_events / experiment_counts[:, np.newaxis, ...], 0)
/tmp/ipykernel_29055/1728990570.py:56: RuntimeWarning: invalid value encountered in divide
  unseen_parameters = np.nan_to_num(unseen_events / experiment_counts, 0)


In [649]:
def update_sender(prior, pack_received, pack_num, pick_num):
    # pack_received is the remaining cards as an np mask vector after the pack/pick being modeled, e.g. 12 cards for p1p1
    
    if pick_num == 12:
        no_no = unseen_parameters[13*pack_num + pick_num, NO_NO]
        no_yes = unseen_parameters[13*pack_num + pick_num, NO_YES]
        yes_yes = unseen_parameters[13*pack_num + pick_num, YES_YES]
    else:
        pack_alsa = np.where(pack_received, alsa, NUM_IN_PACK+1)
        top_received_card = np.argmin(pack_alsa)
        
        no_no = sender_parameters[13*pack_num + pick_num, top_received_card, NO_NO]
        no_yes = sender_parameters[13*pack_num + pick_num, top_received_card, NO_YES]
        yes_yes = sender_parameters[13*pack_num + pick_num, top_received_card, YES_YES]

    # Bayesian updating for part of both sides that becomes "yes" to each distribution function
    yes_update = prior * yes_yes + (1-prior) * no_yes

    # Bayesian updating for the part that remains no
    no_update = (1 - prior) * no_no

    return yes_update / (yes_update + no_update)

def update_receiver(prior, pack_sent, pack_num, pick_num):
    # pack_sent is all the cards sent, empty for pxp1
    
    if pick_num == 0:
        no_no = unseen_parameters[13*pack_num + pick_num, NO_NO]
        no_yes = unseen_parameters[13*pack_num + pick_num, NO_YES]
        yes_yes = unseen_parameters[13*pack_num + pick_num, YES_YES]
    else:
        pack_alsa = np.where(pack_sent, alsa, NUM_IN_PACK + 1)
        top_sent_card = np.argmin(pack_alsa)
            
        yes_yes = receiver_parameters[13*pack_num + pick_num, top_sent_card, YES_YES]
        no_yes = receiver_parameters[13*pack_num + pick_num, top_sent_card, NO_YES]
        no_no = receiver_parameters[13*pack_num + pick_num, top_sent_card, NO_NO]

    # Bayesian updating for part of both sides that becomes "yes" to each distribution function
    yes_update = prior * yes_yes + (1-prior) * no_yes

    # Bayesian updating for the part that remains no
    no_update = (1 - prior) * no_no

    return yes_update / (yes_update + no_update)
    
    

In [650]:
def get_pick_num(row):
    return int(row[PICKNUM_IDX])

def get_pack_num(row):
    return int(row[PACKNUM_IDX])

LEFT = 'left'
RIGHT = 'right'

def update_as_if_sender(prior, row):
    pack = card_vector(PACK_IDX, row)
    pack[names.index(row[PICK_IDX])] -= 1
    return update_sender(prior, pack, get_pack_num(row), get_pick_num(row))

def update_as_if_receiver(prior, row):
    pack = card_vector(PACK_IDX, row)
    return update_receiver(prior, pack, get_pack_num(row), get_pick_num(row))

def simulate_full_pack_as_if_sender(prior, pack_rows, full_print=True):
    for row in pack_rows:
        if full_print:
            print('Updating distribution as if this pack was sent from the modeled user, less the picked card')
            print_pack(row)
        prior = update_as_if_sender(prior, row)
        if full_print:
            print_pool_dist(prior)
    return prior

def simulate_full_pack_as_if_receiver(prior, pack_rows, full_print=True):
    for row in pack_rows:
        if full_print:
            print('Updating distribution as if this pack was received by the modeled user from your seat (or opened)')
            print_pack(row)
        prior = update_as_if_receiver(prior, row)
        if full_print:
            print_pool_dist(prior)
    return prior

def simulate_on_test_draft(draft_rows, as_seat, full_print=True):
    assert len(draft_rows) == 3 * NUM_IN_PACK
    prior = np.zeros((5,40))

    if as_seat == LEFT:
        prior = simulate_full_pack_as_if_sender(prior, draft_rows[0 : NUM_IN_PACK], full_print)
        prior = simulate_full_pack_as_if_receiver(prior, draft_rows[NUM_IN_PACK : 2 * NUM_IN_PACK], full_print)
        prior = simulate_full_pack_as_if_sender(prior, draft_rows[2 * NUM_IN_PACK : 3 * NUM_IN_PACK], full_print)
    else:
        prior = simulate_full_pack_as_if_receiver(prior, draft_rows[0 : NUM_IN_PACK], full_print)
        prior = simulate_full_pack_as_if_sender(prior, draft_rows[NUM_IN_PACK : 2 * NUM_IN_PACK], full_print)
        prior = simulate_full_pack_as_if_receiver(prior, draft_rows[2 * NUM_IN_PACK : 3 * NUM_IN_PACK], full_print)
    if full_print:
        row = draft_rows[-1]
        print('Actual final pool:')
        print_pool_dist(pip_counts_to_matrix(pool_pips(card_vector(POOL_IDX, row))).astype(np.double))
       
    return prior

In [659]:
test_num = 3
test_draft = test_rows[test_num * 3 * NUM_IN_PACK: (test_num+1) * 3 * NUM_IN_PACK]
dist = simulate_on_test_draft(test_draft, LEFT)

Updating distribution as if this pack was sent from the modeled user, less the picked card
Draft Id: 3efbc1bbc1424ea4925321c339048d21, Pack 0, Pick 0
--------
No Witnesses (ALSA: 2.41)
Coerced to Kill (ALSA: 3.47)
And 11 other cards
--------
Picked: Coerced to Kill

********
 (0.47 White)
 (0.29 Red)
 (0.26 Green)
 (0.18 Blue)
 (0.15 Black)
1.35 total
********

Updating distribution as if this pack was sent from the modeled user, less the picked card
Draft Id: 3efbc1bbc1424ea4925321c339048d21, Pack 0, Pick 1
--------
Long Goodbye (ALSA: 2.78)
Shock (ALSA: 2.90)
And 10 other cards
--------
Picked: A Killer Among Us

********
W (0.95 White)
R (0.60 Red)
G (0.51 Green)
 (0.43 Blue)
 (0.16 Black)
2.64 total
********

Updating distribution as if this pack was sent from the modeled user, less the picked card
Draft Id: 3efbc1bbc1424ea4925321c339048d21, Pack 0, Pick 2
--------
Person of Interest (ALSA: 3.73)
Kraul Whipcracker (ALSA: 4.40)
And 9 other cards
--------
Picked: Kraul Whipcracker

*

In [556]:
alsa2[names.index('Makeshift Binding')]

2.4312264320344767

In [555]:
alsa2 = last_seen / counts

/tmp/ipykernel_29055/1484912903.py:1: RuntimeWarning: invalid value encountered in divide
  alsa2 = last_seen / counts


In [ ]:
alsa